# 🏥 YOLO Biomedical Object Detection - Complete Training Pipeline

This notebook provides a complete walkthrough for training YOLO models on biomedical data with CSV annotations.

## 📋 What this notebook covers:
1. **Data Loading & Exploration** - Load and visualize your CSV-annotated images
2. **Data Preparation** - Convert CSV annotations to YOLO format
3. **Dataset Creation** - Create train/validation/test splits
4. **Model Training** - Train YOLO v9+ model with custom data
5. **Model Evaluation** - Evaluate performance and visualize results
6. **Inference** - Run predictions on new images

## 🎯 Prerequisites:
- Images in JPG/PNG format
- CSV files with object locations (x, y, width, height, class)
- Python environment with required packages

Let's get started! 🚀


## 📦 Step 1: Install and Import Required Packages


In [1]:
# Install required packages (run this cell first)
!pip install ultralytics torch torchvision opencv-python pillow numpy matplotlib seaborn pandas scikit-learn tqdm pyyaml

# Import all required libraries
import os
import sys
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import yaml
from PIL import Image, ImageDraw, ImageFont
import shutil
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# YOLO imports
from ultralytics import YOLO
import torch

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

print("✅ All packages imported successfully!")
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🔧 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔧 GPU: {torch.cuda.get_device_name(0)}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 29.9 MB/s eta 0:00:0000:0100:01
WARNING ⚠️ torchvision==0.15 is incompatible with torch==2.6.
Run 'pip install torchvision==0.21' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/simpleai/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ All packages imported successfully!
🔧 PyTorch version: 2.6.0
🔧 CUDA available: False


## 📁 Step 2: Data Loading and Exploration

First, let's load and explore your CSV-annotated data. We'll create a data loader that can handle your specific CSV format.


In [3]:
class CSVDataLoader:
    """
    Data loader for CSV-annotated biomedical images
    Supports various CSV formats commonly used in biomedical datasets
    """
    
    def __init__(self, data_dir, image_extensions=['.jpg', '.jpeg', '.png', '.bmp', '.tiff']):
        self.data_dir = Path(data_dir)
        self.image_extensions = image_extensions
        self.images = []
        self.annotations = {}
        self.class_mapping = {}
        
    def load_data(self):
        """Load all images and their corresponding CSV annotations"""
        print(f"🔍 Loading data from: {self.data_dir}")
        
        # Find all image files
        for ext in self.image_extensions:
            self.images.extend(list(self.data_dir.glob(f"*{ext}")))
            self.images.extend(list(self.data_dir.glob(f"*{ext.upper()}")))
        
        self.images = sorted(self.images)
        print(f"📸 Found {len(self.images)} images")
        
        # Load CSV annotations
        csv_files = list(self.data_dir.glob("*.csv"))
        print(f"📄 Found {len(csv_files)} CSV files")
        
        for csv_file in csv_files:
            self._load_csv_annotations(csv_file)
        
        print(f"✅ Loaded annotations for {len(self.annotations)} images")
        return self.images, self.annotations
    
    def _load_csv_annotations(self, csv_file):
        """Load annotations from a CSV file"""
        try:
            df = pd.read_csv(csv_file)
            print(f"📊 CSV columns: {list(df.columns)}")
            
            # Try to detect the format automatically
            format_type = self._detect_csv_format(df)
            print(f"🔍 Detected format: {format_type}")
            
            # Process annotations based on format
            if format_type == 'filename_based':
                self._process_filename_based_csv(df)
            elif format_type == 'image_based':
                self._process_image_based_csv(df)
            else:
                print(f"⚠️ Unknown CSV format in {csv_file.name}")
                
        except Exception as e:
            print(f"❌ Error loading {csv_file.name}: {str(e)}")
    
    def _detect_csv_format(self, df):
        """Detect the CSV annotation format"""
        columns = [col.lower() for col in df.columns]
        
        # Check for filename-based format
        if any('filename' in col or 'image' in col for col in columns):
            return 'filename_based'
        
        # Check for image-based format (one row per image)
        if any('x' in col and 'y' in col for col in columns):
            return 'image_based'
        
        return 'unknown'
    
    def _process_filename_based_csv(self, df):
        """Process CSV where each row represents one annotation"""
        # Common column name mappings
        col_mapping = {
            'filename': ['filename', 'image', 'image_name', 'file'],
            'class': ['class', 'label', 'category', 'class_name'],
            'x': ['x', 'x_min', 'xmin', 'left'],
            'y': ['y', 'y_min', 'ymin', 'top'],
            'width': ['width', 'w'],
            'height': ['height', 'h']
        }
        
        # Find actual column names
        actual_cols = {}
        for key, possible_names in col_mapping.items():
            for col in df.columns:
                if col.lower() in possible_names:
                    actual_cols[key] = col
                    break
        
        print(f"📋 Column mapping: {actual_cols}")
        
        # Group by filename
        for filename, group in df.groupby(actual_cols['filename']):
            annotations = []
            for _, row in group.iterrows():
                class_name = str(row[actual_cols['class']])
                
                # Create class mapping
                if class_name not in self.class_mapping:
                    self.class_mapping[class_name] = len(self.class_mapping)
                
                annotation = {
                    'class_id': self.class_mapping[class_name],
                    'class_name': class_name,
                    'x': float(row[actual_cols['x']]),
                    'y': float(row[actual_cols['y']]),
                    'width': float(row[actual_cols['width']]),
                    'height': float(row[actual_cols['height']])
                }
                annotations.append(annotation)
            
            self.annotations[filename] = annotations
    
    def _process_image_based_csv(self, df):
        """Process CSV where each row represents one image with multiple objects"""
        # This format is less common but we'll handle it
        print("⚠️ Image-based CSV format detected - this may need custom processing")
        # Add custom logic here based on your specific format
    
    def get_class_mapping(self):
        """Get the class name to ID mapping"""
        return self.class_mapping
    
    def get_dataset_info(self):
        """Get dataset statistics"""
        total_annotations = sum(len(anns) for anns in self.annotations.values())
        class_counts = {}
        
        for anns in self.annotations.values():
            for ann in anns:
                class_name = ann['class_name']
                class_counts[class_name] = class_counts.get(class_name, 0) + 1
        
        return {
            'total_images': len(self.images),
            'total_annotations': total_annotations,
            'num_classes': len(self.class_mapping),
            'class_counts': class_counts,
            'avg_annotations_per_image': total_annotations / len(self.images) if self.images else 0
        }

# Initialize the data loader
# Update this path to your data directory
DATA_DIR = "./your_data"  # Change this to your actual data path

# Create data loader
data_loader = CSVDataLoader(DATA_DIR)

# Load the data
images, annotations = data_loader.load_data()

# Get dataset information
dataset_info = data_loader.get_dataset_info()
print(f"\n📊 Dataset Information:")
print(f"   Total images: {dataset_info['total_images']}")
print(f"   Total annotations: {dataset_info['total_annotations']}")
print(f"   Number of classes: {dataset_info['num_classes']}")
print(f"   Average annotations per image: {dataset_info['avg_annotations_per_image']:.2f}")

print(f"\n📋 Class distribution:")
for class_name, count in dataset_info['class_counts'].items():
    print(f"   {class_name}: {count}")

print(f"\n🔢 Class mapping:")
class_mapping = data_loader.get_class_mapping()
for class_name, class_id in class_mapping.items():
    print(f"   {class_id}: {class_name}")


🔍 Loading data from: your_data
📸 Found 0 images
📄 Found 0 CSV files
✅ Loaded annotations for 0 images

📊 Dataset Information:
   Total images: 0
   Total annotations: 0
   Number of classes: 0
   Average annotations per image: 0.00

📋 Class distribution:

🔢 Class mapping:


## 📊 Step 3: Data Visualization and Exploration

Let's visualize some of your data to understand the annotations and ensure everything is loaded correctly.


In [4]:
def visualize_annotations(image_path, annotations, class_mapping, max_images=4):
    """
    Visualize images with their annotations
    """
    # Create a figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    # Get a sample of images with annotations
    annotated_images = [(img, anns) for img, anns in zip(images, [annotations.get(img.name, []) for img in images]) if anns]
    
    for i, (img_path, img_annotations) in enumerate(annotated_images[:max_images]):
        if i >= len(axes):
            break
            
        # Load image
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Draw annotations
        for ann in img_annotations:
            x, y, w, h = ann['x'], ann['y'], ann['width'], ann['height']
            class_name = ann['class_name']
            
            # Draw bounding box
            cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), (255, 0, 0), 2)
            
            # Draw class label
            cv2.putText(image, class_name, (int(x), int(y - 10)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        # Display image
        axes[i].imshow(image)
        axes[i].set_title(f"{img_path.name}\n{len(img_annotations)} objects")
        axes[i].axis('off')
    
    # Hide unused subplots
    for i in range(len(annotated_images), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

def plot_class_distribution(class_counts):
    """Plot the distribution of classes"""
    plt.figure(figsize=(10, 6))
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    
    bars = plt.bar(classes, counts, color='skyblue', edgecolor='navy', alpha=0.7)
    plt.title('Class Distribution in Dataset', fontsize=16, fontweight='bold')
    plt.xlabel('Class Names', fontsize=12)
    plt.ylabel('Number of Annotations', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def plot_annotation_statistics(dataset_info):
    """Plot various annotation statistics"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. Class distribution
    classes = list(dataset_info['class_counts'].keys())
    counts = list(dataset_info['class_counts'].values())
    axes[0].pie(counts, labels=classes, autopct='%1.1f%%', startangle=90)
    axes[0].set_title('Class Distribution (Pie Chart)')
    
    # 2. Total statistics
    stats = ['Total Images', 'Total Annotations', 'Number of Classes']
    values = [dataset_info['total_images'], dataset_info['total_annotations'], dataset_info['num_classes']]
    bars = axes[1].bar(stats, values, color=['lightcoral', 'lightgreen', 'lightblue'])
    axes[1].set_title('Dataset Statistics')
    axes[1].set_ylabel('Count')
    
    # Add value labels
    for bar, value in zip(bars, values):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    str(value), ha='center', va='bottom', fontweight='bold')
    
    # 3. Average annotations per image
    axes[2].bar(['Avg Annotations/Image'], [dataset_info['avg_annotations_per_image']], 
               color='orange', alpha=0.7)
    axes[2].set_title('Average Annotations per Image')
    axes[2].set_ylabel('Count')
    axes[2].text(0, dataset_info['avg_annotations_per_image'] + 0.1, 
                f"{dataset_info['avg_annotations_per_image']:.2f}", 
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualize the data
if len(images) > 0 and len(annotations) > 0:
    print("📊 Visualizing sample images with annotations...")
    visualize_annotations(images[0], annotations, class_mapping)
    
    print("\n📈 Plotting class distribution...")
    plot_class_distribution(dataset_info['class_counts'])
    
    print("\n📊 Plotting dataset statistics...")
    plot_annotation_statistics(dataset_info)
else:
    print("⚠️ No data loaded. Please check your DATA_DIR path and ensure you have images and CSV files.")


⚠️ No data loaded. Please check your DATA_DIR path and ensure you have images and CSV files.


## 🔄 Step 4: Convert CSV Annotations to YOLO Format

Now we'll convert your CSV annotations to YOLO format and create the proper directory structure for training.


In [5]:
class YOLODataConverter:
    """
    Convert CSV annotations to YOLO format and create dataset structure
    """
    
    def __init__(self, images, annotations, class_mapping, output_dir="./yolo_dataset"):
        self.images = images
        self.annotations = annotations
        self.class_mapping = class_mapping
        self.output_dir = Path(output_dir)
        
    def create_yolo_structure(self):
        """Create YOLO dataset directory structure"""
        directories = [
            'images/train',
            'images/val',
            'images/test',
            'labels/train',
            'labels/val',
            'labels/test'
        ]
        
        for directory in directories:
            full_path = self.output_dir / directory
            full_path.mkdir(parents=True, exist_ok=True)
            print(f"✅ Created directory: {full_path}")
    
    def convert_to_yolo_format(self, train_split=0.7, val_split=0.2, test_split=0.1):
        """
        Convert CSV annotations to YOLO format and split data
        """
        print(f"🔄 Converting annotations to YOLO format...")
        print(f"📊 Split ratios - Train: {train_split}, Val: {val_split}, Test: {test_split}")
        
        # Create directory structure
        self.create_yolo_structure()
        
        # Get images with annotations
        annotated_images = []
        for img_path in self.images:
            img_name = img_path.name
            if img_name in self.annotations:
                annotated_images.append((img_path, self.annotations[img_name]))
        
        print(f"📸 Processing {len(annotated_images)} annotated images...")
        
        # Split data
        train_data, temp_data = train_test_split(
            annotated_images, 
            train_size=train_split, 
            random_state=42
        )
        
        val_data, test_data = train_test_split(
            temp_data, 
            train_size=val_split/(val_split + test_split), 
            random_state=42
        )
        
        print(f"📊 Data split:")
        print(f"   Train: {len(train_data)} images")
        print(f"   Validation: {len(val_data)} images")
        print(f"   Test: {len(test_data)} images")
        
        # Process each split
        splits = {
            'train': train_data,
            'val': val_data,
            'test': test_data
        }
        
        for split_name, split_data in splits.items():
            print(f"\n🔄 Processing {split_name} split...")
            self._process_split(split_name, split_data)
        
        # Create dataset.yaml
        self._create_dataset_yaml()
        
        print(f"\n✅ YOLO dataset created successfully in: {self.output_dir}")
    
    def _process_split(self, split_name, split_data):
        """Process a single data split"""
        for img_path, img_annotations in split_data:
            # Copy image
            img_dest = self.output_dir / f"images/{split_name}" / img_path.name
            shutil.copy2(img_path, img_dest)
            
            # Convert annotations to YOLO format
            yolo_annotations = self._convert_annotations_to_yolo(img_path, img_annotations)
            
            # Save YOLO labels
            label_dest = self.output_dir / f"labels/{split_name}" / f"{img_path.stem}.txt"
            self._save_yolo_labels(yolo_annotations, label_dest)
    
    def _convert_annotations_to_yolo(self, img_path, annotations):
        """Convert CSV annotations to YOLO format"""
        # Load image to get dimensions
        image = cv2.imread(str(img_path))
        img_height, img_width = image.shape[:2]
        
        yolo_annotations = []
        
        for ann in annotations:
            # Get absolute coordinates
            x_abs = ann['x']
            y_abs = ann['y']
            w_abs = ann['width']
            h_abs = ann['height']
            
            # Convert to YOLO format (normalized center coordinates and dimensions)
            center_x = (x_abs + w_abs / 2) / img_width
            center_y = (y_abs + h_abs / 2) / img_height
            width = w_abs / img_width
            height = h_abs / img_height
            
            # Ensure coordinates are within [0, 1]
            center_x = max(0, min(1, center_x))
            center_y = max(0, min(1, center_y))
            width = max(0, min(1, width))
            height = max(0, min(1, height))
            
            yolo_annotation = [
                ann['class_id'],
                center_x,
                center_y,
                width,
                height
            ]
            
            yolo_annotations.append(yolo_annotation)
        
        return yolo_annotations
    
    def _save_yolo_labels(self, yolo_annotations, output_file):
        """Save YOLO format labels to file"""
        with open(output_file, 'w') as f:
            for ann in yolo_annotations:
                line = ' '.join([str(x) for x in ann])
                f.write(line + '\n')
    
    def _create_dataset_yaml(self):
        """Create dataset.yaml file for YOLO training"""
        # Sort class names by their IDs
        sorted_classes = sorted(self.class_mapping.items(), key=lambda x: x[1])
        class_names = [name for name, _ in sorted_classes]
        
        yaml_content = {
            'path': str(self.output_dir.absolute()),
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'nc': len(self.class_mapping),
            'names': class_names
        }
        
        yaml_path = self.output_dir / 'dataset.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)
        
        print(f"✅ Created dataset.yaml: {yaml_path}")
        print(f"📋 Classes: {class_names}")
        
        return yaml_path

# Convert data to YOLO format
if len(images) > 0 and len(annotations) > 0:
    print("🔄 Converting CSV data to YOLO format...")
    
    converter = YOLODataConverter(images, annotations, class_mapping, output_dir="./yolo_dataset")
    converter.convert_to_yolo_format(
        train_split=0.7,
        val_split=0.2, 
        test_split=0.1
    )
    
    print("\n✅ Data conversion completed!")
    print("📁 YOLO dataset structure created in './yolo_dataset'")
else:
    print("⚠️ No data to convert. Please ensure your data is loaded correctly.")


⚠️ No data to convert. Please ensure your data is loaded correctly.


## 🚀 Step 5: Model Training

Now let's train the YOLO model on your converted dataset. We'll use YOLOv9 with biomedical-optimized settings.


In [ ]:
class YOLOTrainer:
    """
    YOLO model trainer with biomedical-optimized settings
    """
    
    def __init__(self, dataset_path, model_name="yolov9c.pt"):
        self.dataset_path = dataset_path
        self.model_name = model_name
        self.model = None
        
    def setup_model(self):
        """Initialize YOLO model"""
        print(f"🔍 Loading YOLO model: {self.model_name}")
        self.model = YOLO(self.model_name)
        print("✅ Model loaded successfully")
        
        # Print model info
        print(f"📊 Model information:")
        print(f"   Model: {self.model_name}")
        print(f"   Task: {self.model.task}")
        print(f"   Classes: {len(self.model.names)}")
        
    def train(self, epochs=100, batch_size=16, img_size=640, device=0, 
              project="biomedical_yolo", name="exp", **kwargs):
        """
        Train the YOLO model with custom parameters
        """
        if self.model is None:
            self.setup_model()
        
        print(f"🚀 Starting training...")
        print(f"📊 Training parameters:")
        print(f"   Epochs: {epochs}")
        print(f"   Batch size: {batch_size}")
        print(f"   Image size: {img_size}")
        print(f"   Device: {device}")
        print(f"   Project: {project}")
        print(f"   Name: {name}")
        
        # Training configuration optimized for biomedical data
        training_config = {
            'data': self.dataset_path,
            'epochs': epochs,
            'imgsz': img_size,
            'batch': batch_size,
            'device': device,
            'project': project,
            'name': name,
            'exist_ok': True,
            'pretrained': True,
            'optimizer': 'auto',
            'verbose': True,
            'seed': 42,
            'deterministic': True,
            'single_cls': False,
            'rect': False,
            'cos_lr': False,
            'close_mosaic': 10,
            'resume': False,
            'amp': True,
            'fraction': 1.0,
            'cache': False,
            'overlap_mask': True,
            'mask_ratio': 4,
            'dropout': 0.0,
            'val': True,
            'plots': True,
            'save_period': 10,
            
            # Biomedical-optimized augmentation settings
            'hsv_h': 0.01,      # Minimal hue change for medical images
            'hsv_s': 0.3,       # Reduced saturation change
            'hsv_v': 0.2,       # Reduced brightness change
            'degrees': 0.0,     # No rotation (preserve orientation)
            'translate': 0.1,   # Small translation
            'scale': 0.3,       # Reduced scaling
            'shear': 0.0,       # No shearing
            'perspective': 0.0, # No perspective distortion
            'flipud': 0.0,      # No vertical flip
            'fliplr': 0.3,      # Reduced horizontal flip
            'mosaic': 0.8,      # Reduced mosaic augmentation
            'mixup': 0.0,       # No mixup for medical images
            
            # Loss weights
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            
            # Learning rate settings
            'lr0': 0.01,
            'lrf': 0.01,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
        }
        
        # Update with any custom parameters
        training_config.update(kwargs)
        
        try:
            # Start training
            results = self.model.train(**training_config)
            
            print("✅ Training completed successfully!")
            return results
            
        except Exception as e:
            print(f"❌ Training failed: {str(e)}")
            return None
    
    def get_training_results(self, results):
        """Extract and display training results"""
        if results is None:
            print("❌ No training results available")
            return
        
        print(f"\n📊 Training Results:")
        print(f"   Best model saved at: {results.save_dir}")
        print(f"   Training completed in: {results.epochs} epochs")
        
        # Get final metrics
        if hasattr(results, 'results_dict'):
            metrics = results.results_dict
            print(f"   Final mAP@0.5: {metrics.get('metrics/mAP50(B)', 'N/A')}")
            print(f"   Final mAP@0.5:0.95: {metrics.get('metrics/mAP50-95(B)', 'N/A')}")
            
            if 'seg' in metrics:
                print(f"   Segmentation mAP@0.5: {metrics.get('metrics/mAP50(S)', 'N/A')}")
                print(f"   Segmentation mAP@0.5:0.95: {metrics.get('metrics/mAP50-95(S)', 'N/A')}")

# Training configuration
TRAINING_CONFIG = {
    'epochs': 100,           # Number of training epochs
    'batch_size': 16,        # Batch size (reduce if you get CUDA out of memory)
    'img_size': 640,         # Input image size
    'device': 0,             # GPU device (use 'cpu' for CPU training)
    'project': 'biomedical_yolo',
    'name': 'csv_training'
}

# Check if dataset exists
dataset_yaml_path = "./yolo_dataset/dataset.yaml"
if os.path.exists(dataset_yaml_path):
    print(f"✅ Dataset found: {dataset_yaml_path}")
    
    # Initialize trainer
    trainer = YOLOTrainer(dataset_yaml_path, model_name="yolov9c.pt")
    
    # Start training
    print("🚀 Starting YOLO training...")
    results = trainer.train(**TRAINING_CONFIG)
    
    # Display results
    trainer.get_training_results(results)
    
    # Save best model path for later use
    if results:
        best_model_path = results.save_dir / "weights" / "best.pt"
        print(f"\n💾 Best model saved at: {best_model_path}")
        
        # Save model path to a file for easy access
        with open("best_model_path.txt", "w") as f:
            f.write(str(best_model_path))
        print("📝 Model path saved to 'best_model_path.txt'")
        
else:
    print(f"❌ Dataset not found: {dataset_yaml_path}")
    print("Please ensure you have run the data conversion step first.")


## 🔍 Step 6: Model Inference and Testing

Let's test our trained model on some sample images to see how it performs.


In [ ]:
class YOLOInference:
    """
    YOLO model inference for testing and prediction
    """
    
    def __init__(self, model_path, conf_threshold=0.25, iou_threshold=0.45):
        self.model_path = model_path
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        self.model = None
        
    def load_model(self):
        """Load the trained YOLO model"""
        print(f"🔍 Loading model from: {self.model_path}")
        self.model = YOLO(self.model_path)
        print("✅ Model loaded successfully")
        
        # Get model info
        print(f"📊 Model information:")
        print(f"   Classes: {list(self.model.names.values())}")
        print(f"   Confidence threshold: {self.conf_threshold}")
        print(f"   IoU threshold: {self.iou_threshold}")
    
    def predict_single_image(self, image_path, save_results=True, output_dir="./inference_results"):
        """Run inference on a single image"""
        if self.model is None:
            self.load_model()
        
        print(f"🔍 Processing image: {image_path}")
        
        # Run inference
        results = self.model(
            image_path,
            conf=self.conf_threshold,
            iou=self.iou_threshold,
            verbose=False
        )
        
        # Process results
        processed_results = self._process_results(results[0], image_path)
        
        # Save results if requested
        if save_results:
            self._save_results(processed_results, image_path, output_dir)
        
        return processed_results
    
    def predict_batch(self, image_dir, save_results=True, output_dir="./inference_results"):
        """Run inference on a batch of images"""
        if self.model is None:
            self.load_model()
        
        print(f"🔍 Processing batch from: {image_dir}")
        
        # Get all image files
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
        image_files = []
        
        for ext in image_extensions:
            image_files.extend(Path(image_dir).glob(f"*{ext}"))
            image_files.extend(Path(image_dir).glob(f"*{ext.upper()}"))
        
        if not image_files:
            print(f"❌ No images found in {image_dir}")
            return []
        
        print(f"📸 Found {len(image_files)} images")
        
        # Process each image
        all_results = []
        for i, image_path in enumerate(image_files):
            print(f"🔄 Processing {i+1}/{len(image_files)}: {image_path.name}")
            try:
                result = self.predict_single_image(str(image_path), save_results, output_dir)
                all_results.append(result)
            except Exception as e:
                print(f"❌ Error processing {image_path.name}: {str(e)}")
                continue
        
        return all_results
    
    def _process_results(self, result, image_path):
        """Process YOLO results into a structured format"""
        processed = {
            'image_path': image_path,
            'image_shape': result.orig_shape,
            'detections': [],
            'summary': {}
        }
        
        # Process detections
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            class_ids = result.boxes.cls.cpu().numpy().astype(int)
            
            for i in range(len(boxes)):
                detection = {
                    'bbox': boxes[i].tolist(),  # [x1, y1, x2, y2]
                    'confidence': float(confidences[i]),
                    'class_id': int(class_ids[i]),
                    'class_name': self.model.names[int(class_ids[i])]
                }
                processed['detections'].append(detection)
        
        # Add summary statistics
        processed['summary'] = {
            'total_detections': len(processed['detections']),
            'classes_detected': list(set([d['class_name'] for d in processed['detections']])),
            'confidence_range': {
                'min': min([d['confidence'] for d in processed['detections']]) if processed['detections'] else 0,
                'max': max([d['confidence'] for d in processed['detections']]) if processed['detections'] else 0
            }
        }
        
        return processed
    
    def _save_results(self, results, image_path, output_dir):
        """Save inference results"""
        os.makedirs(output_dir, exist_ok=True)
        
        # Save JSON results
        base_name = Path(image_path).stem
        json_path = os.path.join(output_dir, f"{base_name}_results.json")
        
        with open(json_path, 'w') as f:
            json.dump(results, f, indent=2)
        
        # Save annotated image
        annotated_image = self._create_annotated_image(image_path, results)
        image_output_path = os.path.join(output_dir, f"{base_name}_annotated.jpg")
        annotated_image.save(image_output_path)
        
        print(f"💾 Results saved to: {output_dir}")
    
    def _create_annotated_image(self, image_path, results):
        """Create annotated image with bounding boxes and labels"""
        # Load image
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        # Try to load a font, fall back to default if not available
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()
        
        # Draw detections
        for detection in results['detections']:
            bbox = detection['bbox']
            class_name = detection['class_name']
            confidence = detection['confidence']
            
            # Draw bounding box
            x1, y1, x2, y2 = bbox
            draw.rectangle([x1, y1, x2, y2], outline='red', width=3)
            
            # Draw label
            label = f"{class_name}: {confidence:.2f}"
            label_bbox = draw.textbbox((0, 0), label, font=font)
            label_width = label_bbox[2] - label_bbox[0]
            label_height = label_bbox[3] - label_bbox[1]
            
            # Draw label background
            draw.rectangle([x1, y1-label_height-5, x1+label_width+10, y1], fill='red')
            
            # Draw label text
            draw.text((x1+5, y1-label_height-2), label, fill='white', font=font)
        
        return image

def visualize_inference_results(results, max_images=4):
    """Visualize inference results"""
    if not results:
        print("No results to visualize")
        return
    
    # Get sample results
    sample_results = results[:max_images]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    for i, result in enumerate(sample_results):
        if i >= len(axes):
            break
        
        # Load original image
        image = cv2.imread(result['image_path'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Draw detections
        for detection in result['detections']:
            bbox = detection['bbox']
            class_name = detection['class_name']
            confidence = detection['confidence']
            
            x1, y1, x2, y2 = [int(coord) for coord in bbox]
            
            # Draw bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            # Draw label
            label = f"{class_name}: {confidence:.2f}"
            cv2.putText(image, label, (x1, y1 - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        
        # Display image
        axes[i].imshow(image)
        axes[i].set_title(f"{Path(result['image_path']).name}\n{result['summary']['total_detections']} detections")
        axes[i].axis('off')
    
    # Hide unused subplots
    for i in range(len(sample_results), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Test inference on trained model
if os.path.exists("best_model_path.txt"):
    with open("best_model_path.txt", "r") as f:
        model_path = f.read().strip()
    
    if os.path.exists(model_path):
        print(f"✅ Found trained model: {model_path}")
        
        # Initialize inference
        inference = YOLOInference(model_path, conf_threshold=0.25, iou_threshold=0.45)
        
        # Test on validation images
        val_images_dir = "./yolo_dataset/images/val"
        if os.path.exists(val_images_dir):
            print("🔍 Running inference on validation images...")
            results = inference.predict_batch(val_images_dir, save_results=True)
            
            if results:
                print(f"\n📊 Inference Results Summary:")
                total_detections = sum(r['summary']['total_detections'] for r in results)
                print(f"   Processed {len(results)} images")
                print(f"   Total detections: {total_detections}")
                print(f"   Average detections per image: {total_detections/len(results):.2f}")
                
                # Visualize results
                print("\n📊 Visualizing inference results...")
                visualize_inference_results(results)
            else:
                print("❌ No inference results generated")
        else:
            print(f"❌ Validation images directory not found: {val_images_dir}")
    else:
        print(f"❌ Model file not found: {model_path}")
else:
    print("❌ No trained model found. Please run the training step first.")


## 📊 Step 7: Model Evaluation and Performance Analysis

Let's evaluate our trained model's performance on the test set and generate comprehensive metrics.


In [ ]:
class YOLOEvaluator:
    """
    Comprehensive YOLO model evaluation
    """
    
    def __init__(self, model_path, dataset_path, conf_threshold=0.25, iou_threshold=0.45):
        self.model_path = model_path
        self.dataset_path = dataset_path
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        self.model = None
        self.class_names = None
        
    def load_model_and_dataset(self):
        """Load model and dataset information"""
        print(f"🔍 Loading model from: {self.model_path}")
        self.model = YOLO(self.model_path)
        
        # Load dataset info
        with open(self.dataset_path, 'r') as f:
            dataset_info = yaml.safe_load(f)
        
        self.class_names = dataset_info['names']
        self.dataset_root = Path(dataset_info['path'])
        
        print(f"✅ Model and dataset loaded successfully")
        print(f"📋 Classes: {self.class_names}")
    
    def evaluate_model(self, output_dir="./evaluation_results"):
        """Run comprehensive model evaluation"""
        if self.model is None:
            self.load_model_and_dataset()
        
        print("🚀 Starting model evaluation...")
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Get test images
        test_images_dir = self.dataset_root / "images/test"
        test_labels_dir = self.dataset_root / "labels/test"
        
        if not test_images_dir.exists():
            print("❌ Test images directory not found")
            return
        
        test_images = list(test_images_dir.glob("*.jpg")) + list(test_images_dir.glob("*.png"))
        
        if not test_images:
            print("❌ No test images found")
            return
        
        print(f"📸 Found {len(test_images)} test images")
        
        # Run predictions and collect ground truth
        all_predictions = []
        all_ground_truth = []
        
        for i, img_path in enumerate(test_images):
            print(f"🔄 Processing {i+1}/{len(test_images)}: {img_path.name}")
            
            # Get ground truth
            gt_path = test_labels_dir / f"{img_path.stem}.txt"
            ground_truth = self._load_ground_truth(gt_path, img_path)
            
            # Run prediction
            predictions = self._run_prediction(img_path)
            
            # Store results
            all_predictions.append(predictions)
            all_ground_truth.append(ground_truth)
        
        # Calculate metrics
        evaluation_results = self._calculate_metrics(all_predictions, all_ground_truth)
        
        # Generate reports
        self._generate_reports(evaluation_results, output_dir)
        
        # Create visualizations
        self._create_visualizations(evaluation_results, output_dir)
        
        print(f"✅ Evaluation completed! Results saved to: {output_dir}")
        return evaluation_results
    
    def _load_ground_truth(self, label_path, image_path):
        """Load ground truth annotations"""
        ground_truth = []
        
        if not label_path.exists():
            return ground_truth
        
        # Load image dimensions
        img = cv2.imread(str(image_path))
        img_height, img_width = img.shape[:2]
        
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id = int(parts[0])
                    center_x = float(parts[1])
                    center_y = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    # Convert to absolute coordinates
                    x1 = (center_x - width/2) * img_width
                    y1 = (center_y - height/2) * img_height
                    x2 = (center_x + width/2) * img_width
                    y2 = (center_y + height/2) * img_height
                    
                    ground_truth.append({
                        'class_id': class_id,
                        'bbox': [x1, y1, x2, y2],
                        'class_name': self.class_names[class_id]
                    })
        
        return ground_truth
    
    def _run_prediction(self, image_path):
        """Run model prediction on single image"""
        results = self.model(
            str(image_path),
            conf=self.conf_threshold,
            iou=self.iou_threshold,
            verbose=False
        )
        
        predictions = []
        if results[0].boxes is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            confidences = results[0].boxes.conf.cpu().numpy()
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for i in range(len(boxes)):
                predictions.append({
                    'class_id': int(class_ids[i]),
                    'bbox': boxes[i].tolist(),
                    'confidence': float(confidences[i]),
                    'class_name': self.class_names[int(class_ids[i])]
                })
        
        return predictions
    
    def _calculate_metrics(self, predictions, ground_truth):
        """Calculate evaluation metrics"""
        print("📊 Calculating evaluation metrics...")
        
        # Initialize metrics storage
        class_metrics = {i: {'tp': 0, 'fp': 0, 'fn': 0} for i in range(len(self.class_names))}
        
        # Calculate IoU and assign predictions to ground truth
        for pred_list, gt_list in zip(predictions, ground_truth):
            # For each ground truth, find best matching prediction
            for gt in gt_list:
                best_iou = 0
                best_pred_idx = -1
                
                for i, pred in enumerate(pred_list):
                    if pred['class_id'] == gt['class_id']:
                        iou = self._calculate_iou(gt['bbox'], pred['bbox'])
                        if iou > best_iou and iou >= self.iou_threshold:
                            best_iou = iou
                            best_pred_idx = i
                
                if best_pred_idx >= 0:
                    class_metrics[gt['class_id']]['tp'] += 1
                    # Remove matched prediction
                    pred_list.pop(best_pred_idx)
                else:
                    class_metrics[gt['class_id']]['fn'] += 1
            
            # Remaining predictions are false positives
            for pred in pred_list:
                class_metrics[pred['class_id']]['fp'] += 1
        
        # Calculate per-class metrics
        evaluation_results = {}
        for class_id in range(len(self.class_names)):
            tp = class_metrics[class_id]['tp']
            fp = class_metrics[class_id]['fp']
            fn = class_metrics[class_id]['fn']
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            evaluation_results[self.class_names[class_id]] = {
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'true_positives': tp,
                'false_positives': fp,
                'false_negatives': fn
            }
        
        # Calculate overall metrics
        total_tp = sum(class_metrics[c]['tp'] for c in range(len(self.class_names)))
        total_fp = sum(class_metrics[c]['fp'] for c in range(len(self.class_names)))
        total_fn = sum(class_metrics[c]['fn'] for c in range(len(self.class_names)))
        
        overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
        
        evaluation_results['overall'] = {
            'precision': overall_precision,
            'recall': overall_recall,
            'f1_score': overall_f1,
            'true_positives': total_tp,
            'false_positives': total_fp,
            'false_negatives': total_fn
        }
        
        return evaluation_results
    
    def _calculate_iou(self, bbox1, bbox2):
        """Calculate Intersection over Union between two bounding boxes"""
        x1_1, y1_1, x2_1, y2_1 = bbox1
        x1_2, y1_2, x2_2, y2_2 = bbox2
        
        # Calculate intersection
        x1_i = max(x1_1, x1_2)
        y1_i = max(y1_1, y1_2)
        x2_i = min(x2_1, x2_2)
        y2_i = min(y2_1, y2_2)
        
        if x2_i <= x1_i or y2_i <= y1_i:
            return 0.0
        
        intersection = (x2_i - x1_i) * (y2_i - y1_i)
        
        # Calculate union
        area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
        area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    def _generate_reports(self, evaluation_results, output_dir):
        """Generate evaluation reports"""
        print("📝 Generating evaluation reports...")
        
        # Save detailed results
        results_file = os.path.join(output_dir, "evaluation_results.json")
        with open(results_file, 'w') as f:
            json.dump(evaluation_results, f, indent=2)
        
        # Create summary report
        summary_file = os.path.join(output_dir, "evaluation_summary.txt")
        with open(summary_file, 'w') as f:
            f.write("YOLO Biomedical Object Detection - Evaluation Summary\n")
            f.write("=" * 60 + "\n\n")
            
            f.write("Overall Performance:\n")
            overall = evaluation_results['overall']
            f.write(f"  Precision: {overall['precision']:.4f}\n")
            f.write(f"  Recall: {overall['recall']:.4f}\n")
            f.write(f"  F1-Score: {overall['f1_score']:.4f}\n")
            f.write(f"  True Positives: {overall['true_positives']}\n")
            f.write(f"  False Positives: {overall['false_positives']}\n")
            f.write(f"  False Negatives: {overall['false_negatives']}\n\n")
            
            f.write("Per-Class Performance:\n")
            for class_name, metrics in evaluation_results.items():
                if class_name != 'overall':
                    f.write(f"  {class_name}:\n")
                    f.write(f"    Precision: {metrics['precision']:.4f}\n")
                    f.write(f"    Recall: {metrics['recall']:.4f}\n")
                    f.write(f"    F1-Score: {metrics['f1_score']:.4f}\n")
                    f.write(f"    TP: {metrics['true_positives']}, FP: {metrics['false_positives']}, FN: {metrics['false_negatives']}\n\n")
        
        print(f"📄 Reports saved to: {output_dir}")
    
    def _create_visualizations(self, evaluation_results, output_dir):
        """Create evaluation visualizations"""
        print("📊 Creating visualizations...")
        
        # 1. Per-class performance comparison
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        class_names = [name for name in evaluation_results.keys() if name != 'overall']
        precisions = [evaluation_results[name]['precision'] for name in class_names]
        recalls = [evaluation_results[name]['recall'] for name in class_names]
        f1_scores = [evaluation_results[name]['f1_score'] for name in class_names]
        
        # Precision
        axes[0].bar(class_names, precisions, color='skyblue')
        axes[0].set_title('Per-Class Precision')
        axes[0].set_ylabel('Precision')
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].set_ylim(0, 1)
        
        # Recall
        axes[1].bar(class_names, recalls, color='lightcoral')
        axes[1].set_title('Per-Class Recall')
        axes[1].set_ylabel('Recall')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].set_ylim(0, 1)
        
        # F1-Score
        axes[2].bar(class_names, f1_scores, color='lightgreen')
        axes[2].set_title('Per-Class F1-Score')
        axes[2].set_ylabel('F1-Score')
        axes[2].tick_params(axis='x', rotation=45)
        axes[2].set_ylim(0, 1)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "per_class_performance.png"), dpi=300, bbox_inches='tight')
        plt.show()
        
        # 2. Overall metrics pie chart
        overall = evaluation_results['overall']
        fig, ax = plt.subplots(figsize=(8, 8))
        
        labels = ['True Positives', 'False Positives', 'False Negatives']
        sizes = [overall['true_positives'], overall['false_positives'], overall['false_negatives']]
        colors = ['lightgreen', 'lightcoral', 'lightblue']
        
        ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        ax.set_title('Overall Detection Results Distribution')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "overall_distribution.png"), dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"📊 Visualizations saved to: {output_dir}")

# Run evaluation if model exists
if os.path.exists("best_model_path.txt"):
    with open("best_model_path.txt", "r") as f:
        model_path = f.read().strip()
    
    dataset_yaml_path = "./yolo_dataset/dataset.yaml"
    
    if os.path.exists(model_path) and os.path.exists(dataset_yaml_path):
        print("📊 Running comprehensive model evaluation...")
        
        evaluator = YOLOEvaluator(
            model_path=model_path,
            dataset_path=dataset_yaml_path,
            conf_threshold=0.25,
            iou_threshold=0.45
        )
        
        evaluation_results = evaluator.evaluate_model("./evaluation_results")
        
        if evaluation_results:
            print("\n📊 Evaluation Summary:")
            overall = evaluation_results['overall']
            print(f"   Overall Precision: {overall['precision']:.4f}")
            print(f"   Overall Recall: {overall['recall']:.4f}")
            print(f"   Overall F1-Score: {overall['f1_score']:.4f}")
            print(f"   Total Detections: {overall['true_positives'] + overall['false_positives']}")
            print(f"   Total Ground Truth: {overall['true_positives'] + overall['false_negatives']}")
    else:
        print("❌ Model or dataset not found. Please ensure training completed successfully.")
else:
    print("❌ No trained model found. Please run the training step first.")


## 🎯 Step 8: Summary and Next Steps

Congratulations! You've successfully completed the entire YOLO training pipeline for biomedical object detection. Here's a summary of what we accomplished:


In [ ]:
# Summary of the complete pipeline
print("🎉 YOLO Biomedical Object Detection - Training Complete!")
print("=" * 60)

print("\n📋 What we accomplished:")
print("✅ 1. Data Loading - Loaded CSV-annotated biomedical images")
print("✅ 2. Data Visualization - Explored and visualized your dataset")
print("✅ 3. Data Conversion - Converted CSV annotations to YOLO format")
print("✅ 4. Dataset Creation - Created train/validation/test splits")
print("✅ 5. Model Training - Trained YOLOv9 with biomedical-optimized settings")
print("✅ 6. Model Inference - Tested predictions on validation data")
print("✅ 7. Model Evaluation - Comprehensive performance analysis")

print("\n📁 Generated files and directories:")
print("📂 yolo_dataset/ - YOLO format dataset")
print("📂 biomedical_yolo/ - Training results and model weights")
print("📂 inference_results/ - Inference results and annotated images")
print("📂 evaluation_results/ - Performance metrics and visualizations")
print("📄 best_model_path.txt - Path to your trained model")

print("\n🔧 Key features of this pipeline:")
print("• Automatic CSV format detection and conversion")
print("• Biomedical-optimized data augmentation")
print("• Comprehensive evaluation metrics")
print("• Visual result analysis")
print("• Easy-to-use inference interface")

print("\n🚀 Next steps you can take:")
print("1. 🔄 Retrain with different parameters:")
print("   - Adjust epochs, batch size, or learning rate")
print("   - Try different YOLO model sizes (yolov9n, yolov9s, yolov9m, yolov9l, yolov9x)")
print("   - Experiment with different augmentation settings")

print("\n2. 📊 Improve model performance:")
print("   - Add more training data")
print("   - Balance your dataset classes")
print("   - Fine-tune hyperparameters")
print("   - Use data augmentation techniques")

print("\n3. 🔍 Deploy your model:")
print("   - Use the trained model for inference on new images")
print("   - Integrate into your biomedical analysis pipeline")
print("   - Export to different formats (ONNX, TensorRT)")

print("\n4. 📈 Monitor and iterate:")
print("   - Track performance on new data")
print("   - Retrain with additional data")
print("   - Compare different model architectures")

print("\n💡 Tips for better results:")
print("• Ensure your CSV annotations are accurate")
print("• Use high-quality, diverse training images")
print("• Consider class imbalance in your dataset")
print("• Monitor training metrics to avoid overfitting")
print("• Validate on representative test data")

print("\n🎯 Your trained model is ready for biomedical object detection!")
print("Use the inference functions to predict on new images.")
